In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:85% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [2]:
import pandas as pd
import numpy as np
import os
from datasets import Dataset, load_dataset
from huggingface_hub import notebook_login
from difflib import SequenceMatcher
from bert_score import score
from itertools import chain
from tqdm.notebook import tqdm

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [35]:
df_list = []
dir_name = "../runs/codeparrot-small"
for file in os.listdir(dir_name):
    df_list.append(pd.read_json(f"{dir_name}/{file}", lines=True))

In [36]:
full = pd.concat(df_list)
full = full.reset_index(drop=True)

In [5]:
full

,task_id,prompt,canonical_solution,test,entry_point,cleaned_generation,full_generation,temperature,top_k,top_p,typical_p,do_sample,num_beams,num_beam_groups,num_return_sequences,num_tokens_to_generate
0,HumanEval/2,\n\ndef truncate_number(number: float) -> floa...,return number % 1.0\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",truncate_number,[\n\ndef truncate_number(number: float) -> flo...,[\n\ndef truncate_number(number: float) -> flo...,1.4,50,1.0,0.1,True,1,1,200,128
1,HumanEval/7,from typing import List\n\n\ndef filter_by_sub...,return [x for x in strings if substring in...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",filter_by_substring,[from typing import List\n\n\ndef filter_by_su...,[from typing import List\n\n\ndef filter_by_su...,1.4,50,1.0,0.1,True,1,1,200,128
2,HumanEval/15,"\n\ndef string_sequence(n: int) -> str:\n ""...",return ' '.join([str(x) for x in range(n +...,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",string_sequence,[\n\ndef string_sequence(n: int) -> str:\n ...,[\n\ndef string_sequence(n: int) -> str:\n ...,1.4,50,1.0,0.1,True,1,1,200,128
3,HumanEval/16,\n\ndef count_distinct_characters(string: str)...,return len(set(string.lower()))\n,"\n\nMETADATA = {\n 'author': 'jt',\n 'da...",count_distinct_characters,[\n\ndef count_distinct_characters(string: str...,[\n\ndef count_distinct_characters(string: str...,1.4,50,1.0,0.1,True,1,1,200,128
4,HumanEval/22,"from typing import List, Any\n\n\ndef filter_i...","return [x for x in values if isinstance(x,...","\n\nMETADATA = {\n 'author': 'jt',\n 'da...",filter_integers,"[from typing import List, Any\n\n\ndef filter_...","[from typing import List, Any\n\n\ndef filter_...",1.4,50,1.0,0.1,True,1,1,200,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7025,HumanEval/138,"\ndef is_equal_to_sum_even(n):\n """"""Evaluat...",return n%2 == 0 and n >= 8\n,def check(candidate):\n assert candidate(4)...,is_equal_to_sum_even,"[\ndef is_equal_to_sum_even(n):\n """"""Evalua...","[\ndef is_equal_to_sum_even(n):\n """"""Evalua...",5.0,50,1.0,1.0,False,100,1,100,128
7026,HumanEval/151,\ndef double_the_difference(lst):\n '''\n ...,return sum([i**2 for i in lst if i > 0 and...,def check(candidate):\n\n # Check some simp...,double_the_difference,[\ndef double_the_difference(lst):\n '''\n ...,[\ndef double_the_difference(lst):\n '''\n ...,5.0,50,1.0,1.0,False,100,1,100,128
7027,HumanEval/152,"\ndef compare(game,guess):\n """"""I think we ...","return [abs(x-y) for x,y in zip(game,guess...",def check(candidate):\n\n # Check some simp...,compare,"[\ndef compare(game,guess):\n """"""I think we...","[\ndef compare(game,guess):\n """"""I think we...",5.0,50,1.0,1.0,False,100,1,100,128
7028,HumanEval/157,"\ndef right_angle_triangle(a, b, c):\n '''\...",return a*a == b*b + c*c or b*b == a*a + c*...,def check(candidate):\n\n # Check some simp...,right_angle_triangle,"[\ndef right_angle_triangle(a, b, c):\n '''...","[\ndef right_angle_triangle(a, b, c):\n '''...",5.0,50,1.0,1.0,False,100,1,100,128


In [21]:
ds = Dataset.from_pandas(full)

In [7]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [25]:
ds.push_to_hub('humaneval-sweeps-codeparrot-small')

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/huggingface_hub/hf_api.py:1948: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  warnings.warn(


In [15]:
ds = load_dataset('cakiki/humaneval-sweeps-codeparrot-small', split='train')

Using custom data configuration cakiki--humaneval-sweeps-codeparrot-small-235b3df3c7122257
Reusing dataset parquet (/tf/data/cache/HF/datasets/cakiki___parquet/cakiki--humaneval-sweeps-codeparrot-small-235b3df3c7122257/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


In [16]:
ds

Dataset({
    features: ['task_id', 'prompt', 'canonical_solution', 'test', 'entry_point', 'cleaned_generation', 'full_generation', 'temperature', 'top_k', 'top_p', 'typical_p', 'do_sample', 'num_beams', 'num_beam_groups', 'num_return_sequences', 'num_tokens_to_generate'],
    num_rows: 7030
})

In [21]:
ds = ds.map(lambda x: {'edit_distance': [SequenceMatcher(None, x['canonical_solution'], gen[len(x['prompt']):]).ratio() for gen in x['cleaned_generation']]}, num_proc=12)

#0:   0%|          | 0/586 [00:00<?, ?ex/s]

#1:   0%|          | 0/586 [00:00<?, ?ex/s]

#2:   0%|          | 0/586 [00:00<?, ?ex/s]

#3:   0%|          | 0/586 [00:00<?, ?ex/s]

#5:   0%|          | 0/586 [00:00<?, ?ex/s]

...

#7:   0%|          | 0/586 [00:00<?, ?ex/s]

#8:   0%|          | 0/586 [00:00<?, ?ex/s]

#9:   0%|          | 0/586 [00:00<?, ?ex/s]

#10:   0%|          | 0/585 [00:00<?, ?ex/s]

#11:   0%|          | 0/585 [00:00<?, ?ex/s]

In [4]:
refs = list(chain(*full.apply(lambda x: [x['canonical_solution']]*len(x['cleaned_generation']),axis=1)))
cands = list(chain(*full.apply(lambda x: [g[len(x['prompt']):] for g in x['cleaned_generation']],axis=1)))

In [8]:
import gc

gc.collect()

3153

In [15]:
# [c for c in cands if len(c) == 5] FAILURES

In [16]:
%%time
prec_list = []
rec_list = []
f1_list = []
for r, c in tqdm(zip(np.array_split(refs,1000), np.array_split(cands,1000)), total=1000):
    precision, recall, f1 = score(refs=r.tolist(), 
                                  cands=c.tolist(),
                                  model_type="microsoft/codebert-base", 
                                  num_layers=12,
                                  use_fast_tokenizer=True, 
                                  device='cuda', 
                                  batch_size=64)
    prec_list.extend(precision.tolist())
    rec_list.extend(recall.tolist())
    f1_list.extend(f1.tolist())

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 46min 13s, sys: 10min 24s, total: 56min 38s
Wall time: 2h 22min 3s


In [17]:
import pickle

with open('p.pkl', 'wb') as f:
    pickle.dump(prec_list, f)
    
with open('r.pkl', 'wb') as f:
    pickle.dump(rec_list, f)
    
with open('f.pkl', 'wb') as f:
    pickle.dump(f1_list, f)

In [24]:
pd.Series(prec_list).describe()

count    852850.000000
mean          0.423652
std           0.394634
min           0.000000
25%           0.000000
50%           0.732549
75%           0.789823
max           1.000000
dtype: float64

In [26]:
pd.Series(rec_list).describe()

count    852850.000000
mean          0.447733
std           0.417043
min           0.000000
25%           0.000000
50%           0.772340
75%           0.836671
max           1.000000
dtype: float64

In [25]:
pd.Series(f1_list).describe()

count    852850.000000
mean          0.435143
std           0.405111
min           0.000000
25%           0.000000
50%           0.759093
75%           0.812229
max           1.000000
dtype: float64

In [27]:
from itertools import islice

In [40]:
lengths = list(full['cleaned_generation'].apply(len).values)

In [51]:
 = [np.array(list(islice(prec_iter, length))) for length in lengths]

In [58]:
prec_iter = iter(prec_list)
full['codebert-bert-score-prec'] = [np.array(list(islice(prec_iter, length))) for length in lengths]

In [60]:
rec_iter = iter(rec_list)
full['codebert-bert-score-recall'] = [np.array(list(islice(rec_iter, length))) for length in lengths]

In [61]:
f1_iter = iter(f1_list)
full['codebert-bert-score-f1'] = [np.array(list(islice(f1_iter, length))) for length in lengths]

In [63]:
ds = Dataset.from_pandas(full)

In [64]:
ds.push_to_hub('cakiki/humaneval-sweeps-codeparrot-small')

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/huggingface_hub/hf_api.py:1948: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  warnings.warn(


Deleting unused files from dataset repository:   0%|          | 0/3 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.


In [65]:
ds = ds.map(lambda x: {'edit_distance': [SequenceMatcher(None, x['canonical_solution'], gen[len(x['prompt']):]).ratio() for gen in x['cleaned_generation']]}, num_proc=12)

Parameter 'function'=<function <lambda> at 0x7f7bc1d2daf0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/586 [00:00<?, ?ex/s]

#1:   0%|          | 0/586 [00:00<?, ?ex/s]

#2:   0%|          | 0/586 [00:00<?, ?ex/s]

#3:   0%|          | 0/586 [00:00<?, ?ex/s]

...

#7:   0%|          | 0/586 [00:00<?, ?ex/s]

#8:   0%|          | 0/586 [00:00<?, ?ex/s]

#9:   0%|          | 0/586 [00:00<?, ?ex/s]

#10:   0%|          | 0/585 [00:00<?, ?ex/s]

#11:   0%|          | 0/585 [00:00<?, ?ex/s]

In [66]:
ds.push_to_hub('cakiki/humaneval-sweeps-codeparrot-small')

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/3 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.


In [74]:
np.min(ds[0]['edit_distance'])

0.0

In [75]:
ds.push_to_hub('cakiki/humaneval-codeparrot-small-eval')

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/huggingface_hub/hf_api.py:1948: FutureWarning: `identical_ok` has no effect and is deprecated. It will be removed in 0.11.0.
  warnings.warn(


In [78]:
from evaluate import load
code_eval = load("code_eval")

In [80]:
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [92]:
from evaluate import load
code_eval = load("code_eval")
test_cases = ["assert add(2,3)==5", "assert add(2,3)==5"]
candidates = [["def add(a, b): return a+b", "def add(a,b): return a*b"], ["def add(a, b): return a+b", "def add(a,b): return a*b"]]
pass_at_k, results = code_eval.compute(references=test_cases, predictions=candidates, k=[1, 0.2, 30])
print(pass_at_k)

{'pass@1': 0.5, 'pass@0.2': 0.09999999999999998}


In [ ]:
[1, 10, 20, 30, 40, 50, 60]

In [86]:
ds = ds.map(lambda x: {"reference": "\n" + x["test"] + "\n" + x["entry_point"]}, num_proc=12)

#0:   0%|          | 0/586 [00:00<?, ?ex/s]

#1:   0%|          | 0/586 [00:00<?, ?ex/s]

#2:   0%|          | 0/586 [00:00<?, ?ex/s]

#3:   0%|          | 0/586 [00:00<?, ?ex/s]

#4:   0%|          | 0/586 [00:00<?, ?ex/s]

...

#7:   0%|          | 0/586 [00:00<?, ?ex/s]

#8:   0%|          | 0/586 [00:00<?, ?ex/s]

#9:   0%|          | 0/586 [00:00<?, ?ex/s]

#10:   0%|          | 0/585 [00:00<?, ?ex/s]

#11:   0%|          | 0/585 [00:00<?, ?ex/s]

In [99]:
ds.push_to_hub('cakiki/humaneval-codeparrot-small-eval')

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

In [89]:
print(ds[0]['reference'])




METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate(3.5) == 0.5
    assert abs(candidate(1.33) - 0.33) < 1e-6
    assert abs(candidate(123.456) - 0.456) < 1e-6

truncate_number


In [108]:
print(ds[0]['cleaned_generation'][0])



def truncate_number(number: float) -> float:
    """ Given a positive floating point number, it can be decomposed into
    and integer part (largest integer smaller than given number) and decimals
    (leftover part always smaller than 1).

    Return the decimal part of the number.
    >>> truncate_number(3.5)
    0.5
    """
"""
    pyasn1 - Copyright (C) 2010  Aron Krund
    pedan - pauldakn
    License: BSD

This is part of Psyco 
http://github.com/petramjenny/Psyco 

 This is part of PyOMA project

   pyOMA is free software: you can redistribute it and/or modify
   it under the terms of the GNU General Public License as published by
   the Free Software Foundation, either version 3 of the License, or
   (at your option) any later version.

   pyOMA is distributed


In [3]:
ds = load_dataset('cakiki/humaneval-codeparrot-small-eval', split='train')

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Using custom data configuration cakiki--humaneval-codeparrot-small-eval-9755818be8af2d7a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /tf/data/cache/HF/datasets/cakiki___parquet/cakiki--humaneval-codeparrot-small-eval-9755818be8af2d7a/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


In [5]:
from evaluate import load

In [6]:
code_eval = load('code_eval')

In [9]:
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [11]:
# %%time
# ds = ds.map(lambda x: {'pass_at_k': code_eval.compute(references=[x['reference']], predictions=[x['cleaned_generation']], k=[1,10,100,200], num_workers=28)})